In [1]:
import os
import ipywidgets as widgets
from geopy.geocoders import Nominatim
import plotly.graph_objects as go

import earthkit.plots
import earthkit.data
import matplotlib.pyplot as plt

In [2]:
os.environ['POLYTOPE_ADDRESS'] = 'polytope-test.ecmwf.int'
geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org")

In [3]:
def get_data(location):
    location = geolocator.geocode(location)
    request = {
        "class": "od",
        "stream": "enfo",
        "type": "pf",
        "levtype": "sfc",
        "date": "20240125",
        "time": "0000",
        "expver": "0001",
        "domain": "g",
        "param": "49/164/167", # 228/167/164/123
        "number": "1/to/50",
        "feature" : {
            "type" : "timeseries",
            "points": [[location.latitude, location.longitude]],
            "start": 0,
            "end" : 120,
        },
    }

    return earthkit.data.from_source("polytope", "ecmwf-mars-beta", request)

In [4]:
LOCATIONS = [
    "Reading, UK",
    "Bonn, Germany",
    "Bologna, Italy",
    "Baltimore, MD, USA",
]

TIME_FREQUENCY = "6H"

a = widgets.Dropdown(
    options=LOCATIONS,
    description="Location",
)

def f(location):
    data = get_data(location=location)
    chart = earthkit.plots.Chart()
    chart.box(data, time_frequency=TIME_FREQUENCY)
    chart.line(data, time_frequency=TIME_FREQUENCY, aggregation="mean", line_color="purple")
    chart.show()

ui = widgets.HBox([a])
out = widgets.interactive_output(f, {'location': a})
display(ui, out)
a.value = "Baltimore, MD, USA"